# THE MNIST DATABASE of handwritten digits

The MNIST database of handwritten digits, available from [here](http://yann.lecun.com/exdb/mnist/), has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

Four files are available on this site:
```
train-images-idx3-ubyte.gz:  training set images (9912422 bytes) 
train-labels-idx1-ubyte.gz:  training set labels (28881 bytes) 
t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes) 
t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)
```
These files are not in any standard image format. You have to write your own (very simple) program to read them. However, [keras](https://www.tensorflow.org/api_docs/python/tf/keras) has helper function, [``keras.datasets.mnist.load_data``](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data), that allows you to load this dataset easily. 


In [1]:
import tensorflow as tf
from tensorflow import keras

Let's load and prepare the MNIST dataset:


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

``x_train`` and ``x_test`` are ``uint8`` arrays of grayscale image data with shapes ``(num_samples, 28, 28)``. ``y_train`` and ``y_test`` are ``uint8`` arrays of digit labels (integers in range 0-9) with shapes ``(num_samples,)``.

Let's normalize the image samples from integers (in range 0-255) to floating-point numbers (in range 0.0-1.0):


In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0

Now, let's build the ``Sequential`` model by stacking layers:


In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
])
print(model)

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" scores, one for each class.

Now, we choose an optimizer and loss function for training:


In [5]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

The [``losses.SparseCategoricalCrossentropy``](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss takes a vector of logits and a ``True`` index and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

Now, let's train our model. The ``fit`` method adjusts the model parameters to minimize the loss:


In [6]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 2s 944us/step - loss: 0.2955 - accuracy: 0.9142
Epoch 2/10
1875/1875 [==============================] - 2s 857us/step - loss: 0.1415 - accuracy: 0.9589
Epoch 3/10
1875/1875 [==============================] - 2s 803us/step - loss: 0.1060 - accuracy: 0.9682
Epoch 4/10
1875/1875 [==============================] - 1s 647us/step - loss: 0.0874 - accuracy: 0.9728
Epoch 5/10
1875/1875 [==============================] - 1s 642us/step - loss: 0.0736 - accuracy: 0.9767
Epoch 6/10
1875/1875 [==============================] - 1s 713us/step - loss: 0.0627 - accuracy: 0.9799
Epoch 7/10
1875/1875 [==============================] - 1s 686us/step - loss: 0.0575 - accuracy: 0.9811
Epoch 8/10
1875/1875 [==============================] - 1s 705us/step - loss: 0.0523 - accuracy: 0.9827
Epoch 9/10
1875/1875 [==============================] - 1s 676us/step - loss: 0.0459 - accuracy: 0.9851
Epoch 10/10
1875/1875 [==============================] - 1s 685u

Now, that our model is trained, it's time to evaluate its performance on an unseen dataset. The ``evaluate`` method checks the models performance on the test set.


In [7]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0661 - accuracy: 0.9811


[0.06610903143882751, 0.9811000227928162]

You can see that the image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability instead of logits, you can wrap the trained model, and attach the softmax to it:


In [8]:
probability_model = keras.Sequential([
  model,
  keras.layers.Softmax()
])

In [9]:
probability_model(x_test[:1]).numpy().sum()

1.0

## Convolutional Neural Networks

Now, let's change our model to take advantage of convolutional neural networks. But, we need to add a channels dimension before feeding the data to [``keras.layers.Conv2D``](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D).


In [10]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


Now, let's build the ``Sequential`` model by stacking layers:


In [11]:
conv_model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])
print(conv_model)

Now, we choose an optimizer and loss function for training for the new model:


In [12]:
conv_model.compile(optimizer='adam', 
                   loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                   metrics=['accuracy'])

Now, let's train our new model.


In [13]:
conv_model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.1371 - accuracy: 0.9585
Epoch 2/10
1875/1875 [==============================] - 35s 18ms/step - loss: 0.0424 - accuracy: 0.9864
Epoch 3/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0233 - accuracy: 0.9921
Epoch 4/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0132 - accuracy: 0.9955
Epoch 5/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0109 - accuracy: 0.9963
Epoch 6/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0061 - accuracy: 0.9980
Epoch 7/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0061 - accuracy: 0.9981
Epoch 8/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0052 - accuracy: 0.9983
Epoch 9/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0040 - accuracy: 0.9987
Epoch 10/10
1875/1875 [==============================] - 29s 15m

Now, that our model is trained, let's evaluate its performance on the test dataset.


In [14]:
conv_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0842 - accuracy: 0.9825


[0.08415018022060394, 0.9825000166893005]

You can see that the image classifier is now trained to ~98% accuracy on this dataset.
